# Tarea 4
En esta tarea usted deberá completar el siguiente notebook, en el cual va a implementar el protocolo de firmas de anillo sobre grupos en general. Debe completar exclusivamente las partes marcadas con `##### POR COMPLETAR`.

## Funciones auxiliares
Necesitaremos un test de primalidad para revisar que no nos estén engañando cuando nos entreguen los parámetros del grupo.

In [82]:
import random

def _is_natural_power(n):
    # Para cada posible exponente, hacemos búsqueda binaria de la base
    search_exponent = 2
    
    # Optimiazación: si n no es a ^ k no puede ser a ^ (kr) para ningún
    # r, por lo que sólo probamos con exponentes primos
    avoid_exponents = set()
    
    while pow(2, search_exponent) <= n:
        
        if search_exponent not in avoid_exponents:
            # Usamos búsqueda binaria "creciente" para definir el intervalo
            # inicial
            search_start = 2
            i = 2
            while search_start ** search_exponent < n:
                search_start *= 2
                avoid_exponents.add(search_exponent * i)
                i += 1
                
            upper = search_start
            lower = search_start // 2

            # Búsqueda binaria
            while lower != upper:
                mid = (upper + lower) // 2
                result = pow(mid, search_exponent)
                if result < n:
                    lower = mid + 1
                elif result > n:
                    upper = mid
                else:
                    return True

            # Caso borde en que upper ^ search_exponent era justo n
            if pow(upper, search_exponent) == n:
                return True
            
        search_exponent += 1
    
    return False

In [83]:
def _extended_euclid(a, b):
    if a > b:
        return _extended_euclid_base(a, b)
    return _extended_euclid_base(b, a)

def _extended_euclid_base(a, b):
    prev_r, r = a, b
    prev_s, s = 1, 0
    prev_t, t = 0, 1

    while r != 0:
        q = prev_r // r
        prev_r, r = r, prev_r % r
        prev_s, s = s, prev_s - q * s
        prev_t, t = t, prev_t - q * t

    return prev_r, prev_s, prev_t

In [84]:
def is_probably_prime(n, iterations=100):
    if n == 2:
        return True
    if n % 2 == 0 or n == 1:
        return False
    if _is_natural_power(n):
        return False
    
    found_negative = False
    for i in range(iterations):
        a = random.randint(1, n - 1)
        if _extended_euclid(a, n)[0] > 1:
            return False
        b = pow(a, (n - 1) // 2, n)
        if b == n - 1:
            found_negative = True
        elif b != 1:
            return False
    
    return found_negative

## Un primer grupo
Como un primer ejemplo de grupo, consideramos a los grupos Z<sub>p</sub><sup>\*</sup> vistos en clases. En particular, definimos la clase `ZpStar` cuyas instancias son estos grupos. Para representar a los elementos dentro de Z<sub>p</sub><sup>\*</sup>, en su constructor se crea dinámicamente la clase `Element`.

In [85]:
class ZpStar:
    
    def __init__(self, p):
        if not is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        class Element:
            def __init__(self, value):
                if value < 1 or value > p-1:
                    raise Exception(f"value={value} is not in the range 1,...,{p-1}")
                self.value = value

            # Allows to compare elements with ==
            def __eq__(self, other_element):
                return self.value == other_element.value

            # Allows to operate elements with *
            def __mul__(self, other_element):
                return Element((self.value * other_element.value) % p)

            # Allows to use ** as exponentiation
            def __pow__(self, exponent):
                return Element(pow(self.value, exponent, p))

            # Allows to use str(e) to transform an element into a string
            def __str__(self):
                return str(self.value)

        self.element_class = Element
                
    def get_identity(self):
        return self.get_element(1)
    
    def get_element(self, n):
        return self.element_class(n)

Ahora vamos a definir una clase que inicializa a los participantes del anillo. Cada participante es una instancia de la clase `Signer` que será definida más adelante. De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `RingSetup` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `subgroup_order` no es un número primo, entonces se debe generar una excepción (puede suponer que el valor entregado en `subgroup_order` va a ser un número natural mayor o igual a 1).

In [86]:
import random

class RingSetup:
    def __init__(self, generator, subgroup_order, n_participants, group):
        # Is the order of the generator correct? For this we check that
        # 1. The subgroup order is prime
        # 2. The generator to the power of subgroup_order is the identity
        # 3. The generator is not the identity
        ##### POR COMPLETAR
        if not is_probably_prime(subgroup_order):
            raise Exception("Subroup order is not prime, it's a trap!")
        if generator == group.get_identity():
            raise Exception("The generator is the identity, it's a trap!")
        if generator ** subgroup_order != group.get_identity():
            raise Exception("This is not the real order, it's a trap!")
    
        # Generate a group of participants
        self.participants = [
            Signer(generator, subgroup_order) for _ in range(n_participants)
        ]

        # Store their public keys
        self.public_keys = [x.get_public_key() for x in self.participants]

    def get_public_keys(self):
        return self.public_keys

    def get_random_participant(self):
        return random.choice(self.participants)

Ahora definimos la clase `Signer`.

In [87]:
from hashlib import sha256

class Signer():
    def __init__(self, generator, subgroup_order):
        self.generator = generator
        self.subgroup_order = subgroup_order
        
        # Create and store a secret/public key pair
        self.secret_key = random.randint(1, subgroup_order - 1)
        self.public_key = generator ** self.secret_key

    def get_public_key(self):
        return self.public_key

    # Compute a ring signature for a message and a list of public keys
    def generate_ring_signature(self, message, public_keys):
        # Simplify notation
        q = self.subgroup_order
        g = self.generator
        n = len(public_keys)

        my_r = random.randint(1, q - 1)
        print(f"my r {my_r}")
        my_index = public_keys.index(self.public_key)

        signatures = [0] * len(public_keys)
        challenges = [0] * len(public_keys)
        challenges[(my_index + 1) % n] = int.from_bytes(sha256((str(g ** my_r) + message).encode()).digest()) % q

        for i in range(1, n):
            index = (my_index + i) % n
            signatures[index] = random.randint(1, q - 1)
            R = g ** signatures[index] * public_keys[index] ** (q - challenges[(index)])
            challenges[(index + 1) % n] = int.from_bytes(sha256((str(R) + message).encode()).digest()) % q

        signatures[my_index] = (my_r + challenges[my_index] * self.secret_key) % q
        print(f"s_{my_index}: {signatures[my_index]}")

        random_index = random.randint(0, n - 1)

        return signatures, challenges[random_index], random_index

**Importante:** En la generación de firmas de anillo se debe calcular el hash de un elemento del grupo concatenado con el mensaje. En esta concatenación puede suponer que la forma de transformar los elementos del grupo a string viene dada por `str(n)`. Además, el resultado del hash se debe interpretar como un número, para lo cual debe usar `int.from_bytes(hash.digest())`.

Ahora vamos con la implementación de una clase para verificar las firmas de anillo, llamada `Verifier`:

In [88]:
class Verifier:
    def __init__(self, generator, subgroup_order):
        self.generator = generator
        self.subgroup_order = subgroup_order

    def verify_ring_signature(self, public_keys, message, signatures, challenge, challenge_index):
        # Verify a ring signature
        # simplify notation
        q = self.subgroup_order
        g = self.generator
        n = len(public_keys)
        current_challenge = challenge

        for i in range(1, n + 1):
            index = (challenge_index + i) % n
            prev = (index - 1) % n
            R = g ** signatures[prev] * public_keys[prev] ** (q - current_challenge) 
            current_challenge = int.from_bytes(sha256((str(R) + message).encode()).digest()) % q


        return challenge == current_challenge

Recuerde que estas clases están definidas para cualquier grupo, y por lo tanto se espera que su implementación funcione con una interfaz genérica para interactuar con estos objetos. Por ejemplo, en el siguiente código se utiliza el grupo Z<sub>643</sub><sup>\*</sup>.

In [89]:
if __name__ == "__main__":
    p, generator, order = 643, 4, 107

    group = ZpStar(p)
    g = group.get_element(generator)

    ring = RingSetup(g, order, 2, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:               {message}")
    print(f"Public keys:           {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:            {signatures}")
    print(f"Challenge index:       {challenge_index}")
    print(f"Challenge:             {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:     {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature:   {result}")

Message:               Hello from IIC3253
Public keys:           ['225', '384']
my r 63
s_0: 89
Signatures:            [89, 39]
Challenge index:       0
Challenge:             79
Correct signature:     True
Incorrect signature:   False


Una vez que haya completado las definiciones de las clases para generar y verificar firmas de anillo, el código anterior debe mostrar algo como lo siguiente:
```
Message:               Hello from IIC3253
Public keys:           ['516', '134', '616', '135', '375']
Signatures:            [86, 105, 97, 77, 63]
Challenge index:       2
Challenge:             27
Correct signature:     True
Incorrect signature:   False
``` 

Como un segundo ejemplo considere un grupo Z<sub>p</sub><sup>\*</sup> que es usado en la práctica.

In [90]:
if __name__ == "__main__":
    p = 17125458317614137930196041979257577826408832324037508573393292981642667139747621778802438775238728592968344613589379932348475613503476932163166973813218698343816463289144185362912602522540494983090531497232965829536524507269848825658311420299335922295709743267508322525966773950394919257576842038771632742044142471053509850123605883815857162666917775193496157372656195558305727009891276006514000409365877218171388319923896309377791762590614311849642961380224851940460421710449368927252974870395873936387909672274883295377481008150475878590270591798350563488168080923804611822387520198054002990623911454389104774092183
    generator = 8041367327046189302693984665026706374844608289874374425728797669509435881459140662650215832833471328470334064628508692231999401840332046192569287351991689963279656892562484773278584208040987631569628520464069532361274047374444344996651832979378318849943741662110395995778429270819222431610927356005913836932462099770076239554042855287138026806960470277326229482818003962004453764400995790974042663675692120758726145869061236443893509136147942414445551848162391468541444355707785697825741856849161233887307017428371823608125699892904960841221593344499088996021883972185241854777608212592397013510086894908468466292313
    order = 63762351364972653564641699529205510489263266834182771617563631363277932854227

    group = ZpStar(p)
    g = group.get_element(generator)

    ring = RingSetup(g, order, 5, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:             {message}")
    print(f"Public keys:         {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:          {signatures}")
    print(f"Challenge index:     {challenge_index}")
    print(f"Challenge:           {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:   {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature: {result}")

Message:             Hello from IIC3253
Public keys:         ['5372909809287809801735166720396829319954347629856280024577256236975979092826961090432348356839005431238321182712372181839258080063313848594546856575633510551782608882392399900571393485317269679759572730185263247809675723391484190865210330088500512877797115226468408408495422446085863131515805352953774272477673735326083303367031531296271221381388665358618257726376964180485199420106160112077731511622027380403447712303950650464283749971267608529333073425233442106027630473963611133790088393493328269954612544834417465942063999446100222200783654709559127025052354874038379299458254842300320139430777708188930009447330', '13287602722845743300794009760810819429466363204546505627760488376226981277253602824761684277772248744552606024370878421507257839609393194088437512096731174944239911397970372280990566934009760336912724923141696600278430583390592534326699739594723542774987126185771596358877837174255384337399182821547584171114445478516

**El código anterior debe mostrar algo como lo siguiente:**
```
Message:             Hello from IIC3253
Public keys:         ['2517830512706408905921795855478002593728536695564369541215433752818688200930590557321299749909051472598879120669335098679086201990588247107010097789653908195551279752437632812718275665598981544118144222483417971539126504478450162602157945937397010648489810647065718469382490168669506251046596417881288288171215855554980819427900171741067720532072516680911006949212485467714683404806945579430182054185594443083586850777040120636465968567638599770255939529206928681124967241910901964546789495870764845450024463889404074510190345750685029474604222569514695003846590369682128367051616414901526467695451326379530045592558', '5270207660733698677379444365513408945379377744136899605872564684007387713297275698259389271331930313222248833279818234133325676502997718277365522335802337127213514973087876911959363974028273837680146332749956406312939587316639326361234287134092889550128946946101251194071838292511417343437360053204333113338720434879059546692509087163054679902770733671764169657087871075083734807460448525636150851282642301952677377786268613136393396241253172078947686682915121559772901880565429197026581606172653634629737742972730555848587200334173694239746043842512888925331708616088881856959738377640477242461071750154708445903616', '4290295718095021988675445254121177064863324098662129262783066199469507095390516614588988784651151081847021838069891388637529024358958231522272516661319379263876975933462254915722181953593832102307038557688856863115076519944152396276190121166385820722775366668295276897317161080467748260348839347932318372823312916424994870126430320104002626579135764282125149078013816804252753118899663976237356011515563337011529792802079272290889759995602735879894050605524591966120664658302635005498072511909994294820012378422696663138258009667356360478704456532326341271584849827662837920312336384196576119296369494018430685468680', '13792847608938719465184496332605495096772014492434818726820448973297625496110337310281695366283943689427744865104761363751586849817037871594690900944374765560486045299130773160170404295646136697636055430219586235874176864304625904266871799915419701399962491391759960375485495587129510617070673009476085756962339636286982291420431354276948191820174224482149383930112834886359364335440659720974265296031634791799861703217132358202332570036784042806999307622122074702481782599438455896989610919983313634466780919462516927530851986252215197877236167006541209043383326112119567715847311805192263845904388190616238648430734', '10011078702742748907877554549063737917030769568096501612653682793110146578391243951572559566683051226189421993140073553893701244152062899876172823506318029601710934733311666584459325204769544040648426353181104063982839933074363401080841815187865036706004492352679601943018686523833801129336689360092848382674867507625546496061084567233837778157788044099412181417511241329506870819948620917334468651158286352700116572819839763989519373583221395470872779149373633862617623122324677676370003431239489306454790584454370900581651101310475827028596893256364366744902336888869996807856811203508203406395657423171781292944255']
Signatures:          [10722411777402596653275815292584573392618152762220609600942110785187707759044, 37006871206013953327579217019949343495896645068493381253815289376515652692533, 9428846663174133374369642171000979807568209932573864941983380633043202379046, 7034677916731842110320680178801060993252839947995940335124833952064825645650, 55236473929309756355522738164922161011054131084796464363101483140615989833630]
Challenge index:     2
Challenge:           46334889421771799592061293299405443101194458819117162433548846027888672198309
Correct signature:   True
Incorrect signature: False
```

## Utilizando curvas elípticas
En esta parte usaremos grupos definidos por curvas elípticas. En particular, debe completar la siguiente definición de la clase `EllipticCurve` considerando la definición de curvas elípticas dada en la ecuación (9.2) del la sección 9.3.4 del libro:

Jonathan Katz y Yehuda Lindell. Introduction to Modern Cryptography. Chapman and Hall/CRC,
tercera edición, 2021.

In [95]:
class EllipticCurve:
    def __init__(self, A, B, p):
        if not is_probably_prime(p):
            raise Exception(f"p={p} is not a prime number")
        elif A < 0 or A > p-1:
            raise Exception(f"A={A} is not in the range 0,...,{p-1}")
        elif B < 0 or B > p-1:
            raise Exception(f"B={B} is not in the range 0,...,{p-1}")
        elif (4*pow(A,3) + 27*pow(B,2)) % p == 0:
            raise Exception(f"The determinant is equal to 0 modulo p={p}")             
        class Element:
            def __init__(self, x, y = None):
                if y is None:
                    if not x == 'N':
                        raise Exception(f"x={x} must take value N")
                    self.x = x
                elif x < 0 or x > p-1:
                    raise Exception(f"x={x} is not in the range 0,...,{p-1}")
                elif y < 0 or y > p-1:
                    raise Exception(f"y={y} is not in the range 0,...,{p-1}")
                elif not (pow(y,2) - (pow(x,3) + A*x + B)) % p == 0: 
                    raise Exception(f"Point x={x}, y={y} is outside the elliptic curve")
                else:
                    self.x = x
                    self.y = y

            def __eq__(self, other_element):
                if hasattr(self,'y'):
                    if hasattr(other_element,'y'):
                        return (self.x == other_element.x and self.y == other_element.y)
                    return False
                else:
                    if not hasattr(other_element, 'y'):
                        return (self.x == other_element.x)
                    return False

            def __mul__(self, other_element):
                if (self.x == 'N'):
                    return other_element
                elif (other_element.x == 'N'):
                    return self
                elif not self.x == other_element.x:
                    s = ((self.y - other_element.y)*pow(self.x - other_element.x, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                elif not self.y == other_element.y:
                    return Element('N')
                elif not self.y == 0:
                    s = ((3*pow(self.x, 2) + A)*pow(2*self.y, -1, p)) % p
                    x = (pow(s,2) - self.x - other_element.x) % p
                    y = (s*(self.x - x) - self.y) % p
                    return Element(x,y)
                return Element('N')
                    
            def __pow__(self, exponent):
                if exponent == 0:
                    return Element('N')
                elif exponent == 1:
                    return self
                else:
                    result = Element('N')
                    pot = self
                    while exponent > 0:
                        if exponent % 2 == 1:
                            result = pot * result
                        exponent = exponent // 2
                        pot = pot * pot
                return result            
            
            
            def __str__(self):
                if hasattr(self,'y'):
                    return f"({self.x}, {self.y})"
                return "N"
                    
        self.element_class = Element
                
    def get_identity(self):
        return self.element_class('N')
                    
    def get_element(self, x, y):
        return self.element_class(x,y)

En esta definición de `EllipticCurve`, dado un número primo `p`, cada punto sobre la curva es un par ordenado `(x,y)` con `x` e `y` en el conjunto `{0, ..., p-1}`, excepto por el neutro del grupo que un elemento especial que no necesita notación de par ordenado (ver el libro de Katz & Lindell para una explicación de esto). Por esto el constructor de la clase `EllipticCurve` recibe dos argumentos para representar un par ordenado, y también considera el caso en que `y` no esté definido porque se está utilizando el elemento neutro.

**Importante:** Dado un elemento `g = (x, y)` del grupo, diremos que su interpretación como string es literalmente `(x, y)` (notar el espacio después de la coma). Es decir, la interpretación como string se debe calcular de la forma `f"({self.x}, {self.y})"`.

De la misma forma que para la clase `ZpStar`, su implementación del constructor de la clase `EllipticCurve` debe generar excepciones si los parámetros entregados no son correctos (puede suponer que los tipos de estos parámetros siempre van a ser los correctos). Por ejemplo, si `p` no es un número primo, entonces se debe generar una excepción.

Su definición de la clase `EllipticCurve` va a ser utilizada por la implementación del protocolo de firmas de anillos de la misma forma que para la clase `ZpStar`. Por ejemplo, en el siguiente código se utilizan firmas de anillos sobre la curva elíptica [P-256](https://neuromancer.sk/std/nist/P-256).

In [96]:
if __name__ == "__main__":
    p = 115792089210356248762697446949407573530086143415290314195533631308867097853951
    A = 115792089210356248762697446949407573530086143415290314195533631308867097853948
    B = 41058363725152142129326129780047268409114441015993725554835256314039467401291
    g_x = 48439561293906451759052585252797914202762949526041747995844080717082404635286
    g_y = 36134250956749795798585127919587881956611106672985015071877198253568414405109
    order = 115792089210356248762697446949407573529996955224135760342422259061068512044369

    group = EllipticCurve(A, B, p)
    g = group.get_element(g_x, g_y)
    
    ring = RingSetup(g, order, 5, group)
    signer = ring.get_random_participant()

    message = "Hello from IIC3253"
    print(f"Message:             {message}")
    print(f"Public keys:         {[str(x) for x in ring.get_public_keys()]}")

    signatures, challenge, challenge_index = signer.generate_ring_signature(
        message, ring.get_public_keys()
    )
    print(f"Signatures:          {signatures}")
    print(f"Challenge index:     {challenge_index}")
    print(f"Challenge:           {challenge}")
    
    verifier = Verifier(g, order)
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Correct signature:   {result}")

    signatures[0] = (signatures[0] - 1) % order
    result = verifier.verify_ring_signature(ring.get_public_keys(), message, signatures, challenge, challenge_index)
    print(f"Incorrect signature: {result}")

Message:             Hello from IIC3253
Public keys:         ['(3598407497066439950669138491423938569550390088946766389269715946626286735481, 83779389924371055548242216556298253478796197522600552597296619522215884712464)', '(62018001315755049231659977615295832899780457004437869757165662785875088075884, 105801574659311682213383825423576523992107880330497829412021594347078748090855)', '(36247405782024764799340259755372973516297529415281496755870854908988212531273, 71194965340905737216388107332581034828402750415622227881478236523550035537850)', '(42472779489378114120829378054868609870039822346042883800747075212120805746599, 61245763807127492295913270342516251736972534059703841846406841347276679162030)', '(49511443069808626961006858279366702101651153753044378137788216313527175456547, 57867300172763009626608431061817599809372437815146658086281771380035868121745)']
my r 77418408558476990963226237866912073436393586815662178538196253737630435384753
s_4: 1479857993023605345809864820980637371598

**El código anterior debe mostrar algo como lo siguiente:**

Message:             Hello from IIC3253

Public keys:
['(36228047005606623638394851230173192734851072661042561449319473736621754039520, 20172362822856456120421044511946279043257074756688522685143706143736092565958)', '(58564759885583515378683115765772107948142614786701727763264984960945695330334, 79129698021788006657426468620527515930126766311369828883900257058578878052857)', '(25108858240980601702082892074061491934513031462242737558923411453375449067242, 76425488572625390607396261141639007962966296877847096915671747178845593774896)', '(6701657221200507285541433974699587733893239378552437797081211908168708912451, 97013597839749465309655953073668617169117780346910872542402314282358373971520)', '(113015587168257705614560565034856000733681336982911328860838726980840022492723, 54104620305347279303971506213405538556140509029853428898553160506673533048482)']

Signatures:
[77654962933025577729744678855999034023206810390864799379806439528613544048007, 72513278071731766598881593643721383994593537425755886558349450371764918110449, 28944527442594298721839943005624633745105995076212793147845350194880884108800, 90231818693911206835713485736658448057422079278315344453989541044751933614865, 25344903812783089196604799883051329695362840222494910906975057115807269376447]

Challenge index:     2

Challenge:           86130958050425514850634115208684686516639197236630218119818914977367756079082

Correct signature:   True

Incorrect signature: False

## ¡Buena suerte con la tarea!